## 最优解（x趋近1，y趋近0）

In [1]:
from scipy import optimize
import numpy as np

In [2]:
# 最小化目标函数
def f():
    return lambda x: (x[0]**2 + 3*x[0]*x[1] + 2*x[1]**2 + 4*x[0] + 5*x[1])

In [3]:
# 约束条件
def con():
    # 约束条件 分为eq 和ineq
    #eq表示 函数结果等于0 ； ineq 表示 表达式大于等于0  
    cons = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] - 1},\
              {'type': 'ineq', 'fun': lambda x: x[0]-0.000000000001},\
             {'type': 'ineq', 'fun': lambda x: x[1]-0.000000000001})
    return cons

In [4]:
cons = con()
fun = f()
optimize.minimize(fun, np.array([0.5, 0.5]), method='SLSQP',constraints=cons)

     fun: 5.000000000001999
     jac: array([6.        , 8.00000006])
 message: 'Optimization terminated successfully.'
    nfev: 8
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([1.00000000e+00, 9.99977878e-13])

In [26]:
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import datetime
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from jqdatasdk import *
import warnings
warnings.filterwarnings("ignore")
auth('18610039264', 'zg19491001')

In [35]:
fund_values

,tradedate,nv,chg_fund
0,2011-10-10,1.345,0.000000
1,2011-10-11,1.333,-0.008922
2,2011-10-12,1.366,0.024756
3,2011-10-13,1.384,0.013177
4,2011-10-14,1.381,-0.002168
...,...,...,...
2236,2020-12-16,6.807,0.005911
2237,2020-12-17,6.899,0.013515
2238,2020-12-18,6.942,0.006233
2239,2020-12-21,7.024,0.011812


In [34]:
# 读取数据
df = pd.read_excel("基金净值数据.xlsx",encoding='gbk')
fund_values = df[['日期','复权净值(元)']]
fund_values.columns = ['tradedate','nv']
fund_values['chg_fund'] = fund_values['nv'].pct_change()
fund_values=fund_values.fillna(0)

In [4]:
# 获取价格
def stock_price(sec, period, sday, eday):
    """
    输入 股票代码，开始日期，截至日期
    输出 个股的后复权的开高低收价格
    """
    temp = get_price(sec, start_date=sday, end_date=eday, frequency=period, fields=['open', 'close', 'high', 'low'],
                     skip_paused=True, fq='pre', count=None).reset_index() \
        .rename(columns={'index': 'tradedate'})
    temp['stockcode'] = sec
    return temp

## 计算业绩基准

In [37]:
import math
# 一年期定期存款的日利率
r_f = math.pow(1.0175, 1/252) - 1
#沪深300
hs300 = stock_price('000300.XSHG', '1d', '2011-10-01', '2020-12-22')
hs300['chg_hs300'] = hs300.close.pct_change()
# 国债指数
gz = stock_price('000012.XSHG', '1d', '2011-10-01', '2020-12-22')
gz['chg_gz'] = gz.close.pct_change()
# 业绩基准
b_mark = hs300[['tradedate','chg_hs300']].merge(gz[['tradedate','chg_gz']])
b_mark['chg'] = 0.6 * b_mark['chg_hs300'] + 0.35 * b_mark['chg_gz'] + 0.05 * r_f
b_mark.fillna(0,inplace=True)
b_mark['benchmark'] = (b_mark['chg'] + 1).cumprod()

## 指标计算

In [43]:
# 计算半方差
def Semivariance(x):
    r = x.pct_change().dropna()
    r_mean = np.mean(r)
    lows = [(i-r_mean)**2 for i in r if i <= r_mean]
    return np.sum(lows) / len(lows) * 252

# 计算VaR 
def var(netlist, a=0.05):
    '''
    :param list:netlist
    :return: 95%置信概率的日VAR值
    '''
    r = netlist.pct_change().dropna()
    r_s = pd.Series(r)
    # r_s_p = r_s.rolling(period).apply(np.sum, raw=True)
    r_s = r_s.dropna()
    var = np.quantile(r_s, a, interpolation='linear')
    return (var)

# 计算IR
def ir(netlist1, netlist2):
    r1 = netlist1.pct_change().dropna()
    asset_return = pd.Series(r1)
    r2 = netlist2.pct_change().dropna()
    index_return = pd.Series(r2)
    multiplier = 252
    if asset_return is not None and index_return is not None:
        active_return = asset_return - index_return
        tracking_error = (active_return.std(ddof=1)) * np.sqrt(multiplier)
        asset_annualized_return = multiplier * asset_return.mean()
        index_annualized_return = multiplier * index_return.mean()
        information_ratio = (asset_annualized_return - index_annualized_return) / tracking_error
    else:
        information_ratio = np.nan
    return information_ratio

## 半方差、VAR值、IR值计算结果

In [44]:
b_mark.tradedate = b_mark.tradedate.apply(lambda s: str(s)[:10])
value_data = fund_values.merge(b_mark)
# 计算半方差
Semivar = Semivariance(value_data.nv)
'半方差：', Semivar

('半方差：', 0.046769640625613035)

In [45]:
# 计算VAR
var_1 =  var(value_data.nv, a=0.05)
'VAR值：', var_1

('VAR值：', -0.019217008307242798)

In [47]:
# 计算IR值
IR = ir(value_data.nv, value_data.benchmark)
'IR值：', IR

('IR值：', 1.130152832910393)

## 该基金的择时能力：HM模型

In [49]:
rf_hm = math.pow(1.0175, 1/12) - 1
value_data_2 = value_data[['tradedate','chg_fund','chg']].dropna()
value_data_2['month'] = value_data_2.tradedate.apply(lambda s: str(s)[:7])
hm_data = value_data_2[['month','chg_fund','chg']].groupby('month').apply(lambda x : (1 + x).prod() - 1)
hm_data = hm_data.reset_index()
tmp_x = hm_data.copy()
tmp_x['y'] = tmp_x['chg_fund'] - rf_hm
tmp_x['x1'] = tmp_x['chg'] - rf_hm
tmp_x['x2'] = tmp_x['x1'].apply(lambda s: max(s, 0))
x_tm = tmp_x[['x1', 'x2']]
x_tm = sm.add_constant(x_tm)
model_tm = sm.OLS(tmp_x['y'], x_tm).fit()
[alpha_tm, beta1_tm, beta2_tm] = model_tm.params
[p1_tm, p2_tm, p3_tm] = model_tm.pvalues
# 模型结果
model_tm.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.611    
Dependent Variable: y                AIC:                -440.2490
Date:               2020-12-30 10:08 BIC:                -432.1204
No. Observations:   111              Log-Likelihood:     223.12   
Df Model:           2                F-statistic:        87.31    
Df Residuals:       108              Prob (F-statistic): 2.76e-23 
R-squared:          0.618            Scale:              0.0010801
--------------------------------------------------------------------
            Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const       0.0172     0.0045    3.7969   0.0002    0.0082    0.0261
x1          1.3036     0.1518    8.5891   0.0000    1.0027    1.6044
x2         -0.5000     0.2291   -2.1825   0.0312   -0.9541   -0.0459
------------------------------------------------------------------
Omnibus:               13.679       Durbin-Watson:          1.975 
Prob(Omnibus):         0.001        Jarque-Bera (JB):       42.141
Skew:                  0.200        Prob(JB):               0.000 
Kurtosis:              5.992        Condition No.:          85    
==================================================================

"""